In [3]:
import json
import pandas as pd
import numpy as np
from scipy import stats
import itertools
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from experiments.experiment_config import (
    ALL_LIGHTNING_MODELS,
    DRIVE_AND_ACT_VIEWPOINTS,
    # get_drive_and_act_loaders,
    # two_actors_samples,
    # four_actors_samples,
    # driver_and_act_pretrained_map,
)

sns.set_theme()

In [79]:
subset_percentage = [1, 2, 5, 10, 25, 50, 100]
actors = ['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8']
line_colors = sns.color_palette('muted')[1:]
marker_colors = sns.color_palette('dark')[1:]

def calculate_summary_results(metric):    
    saved_folder = Path(f'saved_lifter_2d_3d_model/rq2/')
    view_point_mean = {}
    view_point_stderr = {}
    for view_point in DRIVE_AND_ACT_VIEWPOINTS:
        view_point_mean[view_point] = {}
        view_point_stderr[view_point] = {}
        for model in ALL_LIGHTNING_MODELS:
            model_name = model.__name__
            simple_model_name = model_name.replace('Lit', '')
            raw_model_mean = []
            raw_model_stderr = []
            ft_model_mean = []
            ft_model_stderr = []
            improv_model_mean = []
            improv_model_stderr = []
            for subset in subset_percentage:
                raw_measure = []
                ft_measure = []
                improv_measure = []
                for actor in actors:
                    raw_training_path = saved_folder / model_name / 'drive_and_act' / view_point / 'single_actor' / f'subset_{subset}' / actor
                    with (raw_training_path / 'test_result.json').open() as f:
                        data = json.loads(f.read())
                        raw_measure.append(pd.DataFrame(data[metric], index=[actor]))

                    transfer_learning_path = saved_folder / model_name / 'transfer_learning' / view_point / 'single_actor' / f'subset_{subset}' / actor
                    with (transfer_learning_path / 'test_result.json').open() as f:
                        data = json.loads(f.read())
                        ft_measure.append(pd.DataFrame(data[metric], index=[actor]))
                raw_model_mean.append(pd.concat(raw_measure).mean(axis=0))
                # raw_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(raw_measure)), ))
                raw_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(raw_measure)), index=raw_measure[0].columns, columns=['PJPE']))
                ft_model_mean.append(pd.concat(ft_measure).mean(axis=0))
                # ft_model_stderr.append(stats.sem(pd.concat(ft_measure)))
                ft_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(ft_measure)), index=ft_measure[0].columns, columns=['std_error']))
                
                for i in range(len(raw_measure)):
                    improv_measure.append(raw_measure[i] - ft_measure[i])
                improv_model_mean.append(pd.concat(improv_measure).mean(axis=0))
                improv_model_stderr.append(pd.DataFrame(stats.sem(pd.concat(improv_measure)), index=improv_measure[0].columns, columns=['std_error']))

            raw_model_mean_df = pd.concat(raw_model_mean, axis=1)
            raw_model_mean_df.columns = subset_percentage
            ft_model_mean_df = pd.concat(ft_model_mean, axis=1)
            ft_model_mean_df.columns = subset_percentage
            improv_model_mean_df = pd.concat(improv_model_mean, axis=1)
            improv_model_mean_df.columns = subset_percentage

            raw_model_stderr_df = pd.concat(raw_model_stderr, axis=1)
            raw_model_stderr_df.columns = subset_percentage
            ft_model_stderr_df = pd.concat(ft_model_stderr, axis=1)
            ft_model_stderr_df.columns = subset_percentage
            improv_model_stderr_df = pd.concat(improv_model_stderr, axis=1)
            improv_model_stderr_df.columns = subset_percentage

            view_point_mean[view_point][simple_model_name] = {
                'raw': raw_model_mean_df,
                'ft': ft_model_mean_df,
                'improv': improv_model_mean_df,
            }
            view_point_stderr[view_point][simple_model_name] = {
                'raw': raw_model_stderr_df,
                'ft': ft_model_stderr_df,
                'improv': improv_model_stderr_df
            }

    return view_point_mean, view_point_stderr

In [120]:
def plot_result(
        metric,
        model,
        view_point,
        view_point_mean,
        view_point_stderr,
        markersize=3,
        elinewidth=1,
        capsize=3,
        legend_font_size=8,
        color_idx=0
    ):
    fig = plt.figure(figsize=(12, 35), constrained_layout = True)
    fig.suptitle(f'{metric} | view: {view_point}')
    model_name = model.__name__
    simple_model_name = model_name.replace('Lit', '')
    item_length = view_point_mean[view_point][simple_model_name]['raw'].index.shape[0]
    subfigs = fig.subfigures(item_length, 1, wspace=0.07)
    for i in range(item_length):
        axes = subfigs[i].subplots(1, 2)
        subfigs[i].suptitle(view_point_mean[view_point][simple_model_name]['raw'].index[i])
        axes[0].set_ylabel('MPJPE [mm]')
        axes[0].set_xlabel('Percentage of samples')
        axes[0].set(xticks=subset_percentage)
        axes[0].set_ylim(0, 150)
        raw_values = view_point_mean[view_point][simple_model_name]['raw'].iloc[i]
        raw_stderr = view_point_stderr[view_point][simple_model_name]['raw'].iloc[i]
        axes[0].plot(
            subset_percentage,
            raw_values,
            color=line_colors[color_idx],
            linestyle='dashed',
            label='No Pretrained'
        )
        axes[0].errorbar(
            subset_percentage,
            raw_values,
            yerr=raw_stderr,
            fmt='s',
            color=marker_colors[color_idx],
            elinewidth=elinewidth,
            capsize=capsize,
            markersize=markersize,
            label='Standard Error'
        )
        ft_values = view_point_mean[view_point][simple_model_name]['ft'].iloc[i]
        ft_stderr = view_point_stderr[view_point][simple_model_name]['ft'].iloc[i]
        axes[0].plot(
            subset_percentage,
            ft_values,
            color=line_colors[color_idx],
            label='Pretrained'
        )
        axes[0].errorbar(
            subset_percentage,
            ft_values,
            yerr=ft_stderr,
            fmt='o',
            elinewidth=elinewidth,
            capsize=capsize,
            markersize=markersize,
            color=marker_colors[color_idx],
            label='Standard Error'
        )
        axes[0].legend(prop={'size': legend_font_size})

        improv_values = view_point_mean[view_point][simple_model_name]['improv'].iloc[i]
        improv_stderr = view_point_stderr[view_point][simple_model_name]['improv'].iloc[i]

        axes[1].step(
            subset_percentage,
            improv_values,
            color=line_colors[color_idx],
            label='Reduction by pretraining',
            where='post'
        )
        axes[1].errorbar(
            subset_percentage,
            improv_values,
            yerr=improv_stderr,
            fmt='o',
            elinewidth=elinewidth,
            capsize=capsize,
            markersize=markersize,
            color=marker_colors[color_idx],
            label='Standard Error'
        )
        axes[1].set_ylabel('MPJPE [mm]')
        axes[1].set_xlabel('Percentage of samples')
        axes[1].set(xticks=subset_percentage)
        axes[1].set_ylim(0, 30)
        axes[1].legend(prop={'size': legend_font_size})

In [ ]:
def get_summarize_table(view_point, simple_model_name, view_point_mean, sort_index=False):
    columns = view_point_mean['a_column_co_driver']['JointFormer']['raw'].index.tolist()
    if sort_index:
        columns = sorted(columns)
    columns = list(itertools.chain(*([[f'{i}', f'{i} (pretrain)'] for i in columns])))
    df = pd.concat(
        [
            view_point_mean[view_point][simple_model_name]['raw'].T,
            view_point_mean[view_point][simple_model_name]['ft'].T.add_suffix(' (pretrain)') 
        ], axis=1
    )[columns].T
    return df

def print_table(df):
    with pd.option_context('float_format', '{:.2f}'.format, 'display.max_row', None):
        display(df)

In [174]:
view_point_mean, view_point_stderr = calculate_summary_results('pjpe')
print_table(get_summarize_table(DRIVE_AND_ACT_VIEWPOINTS[0], ALL_LIGHTNING_MODELS[0].__name__.replace('Lit', ''), view_point_mean))

,1,2,5,10,25,50,100
nose,425.01,195.30,74.13,55.70,56.91,52.65,51.87
nose w/ (pretrain),60.16,51.47,50.66,48.25,51.56,50.21,49.11
left_eye,407.99,195.93,74.30,53.15,55.24,50.40,49.23
left_eye w/ (pretrain),54.12,51.34,48.52,46.82,49.56,47.99,46.87
right_eye,365.29,205.95,75.18,50.22,51.57,48.71,48.11
right_eye w/ (pretrain),55.33,47.99,46.67,45.41,47.97,46.57,45.66
left_ear,166.87,76.19,36.17,20.92,17.84,14.61,14.28
left_ear w/ (pretrain),17.43,14.71,14.71,13.15,13.45,13.70,13.91
right_ear,414.66,199.97,69.87,42.46,42.92,37.78,38.74
right_ear w/ (pretrain),39.07,37.00,38.32,36.45,36.47,35.87,36.12


In [176]:
view_point_mean, view_point_stderr = calculate_summary_results('activity_mpjpe')
print_table(get_summarize_table(DRIVE_AND_ACT_VIEWPOINTS[0], ALL_LIGHTNING_MODELS[0].__name__.replace('Lit', ''), view_point_mean, sort_index=True))

,1,2,5,10,25,50,100
closing_bottle,435.15,236.03,109.42,88.91,85.74,85.65,84.73
closing_bottle w/ (pretrain),90.49,78.81,84.13,84.56,82.73,84.05,82.53
closing_door_inside,533.77,303.91,162.51,139.59,137.22,135.44,138.23
closing_door_inside w/ (pretrain),138.88,138.31,141.13,137.81,129.43,131.93,132.14
closing_laptop,453.63,250.27,136.97,122.25,122.31,120.43,121.91
closing_laptop w/ (pretrain),131.25,116.44,127.94,125.28,120.73,119.05,115.99
drinking,415.57,215.45,99.92,79.40,76.63,76.69,77.23
drinking w/ (pretrain),83.92,79.44,71.83,72.95,73.86,73.01,73.90
eating,384.81,198.57,88.32,66.99,66.96,63.73,62.88
eating w/ (pretrain),76.38,66.90,65.13,64.14,63.05,62.20,60.78
